In [131]:
from astropy.io import fits
import numpy as np
from astropy.table import Table, Column

In [132]:
ra = [[] for _ in range(10)]
dec = [[] for _ in range(10)]
flux = [[] for _ in range(10)]
freq = 182.435

source_i = 0
initial_i = 0
count_i = 0

file_path = '/Users/z3543600/Downloads/flux_data_fixed/flux_data_FWHM:40_ker=90_rf.txt'

with open(file_path, 'r') as file:
    for line in file:
        line = line.strip()
        if line.lower().startswith('source'):
            if initial_i != 0:
                source_i += 1
                continue
            initial_i += 1
            continue
        
        array = line.split(',')

        if float(array[2]) != 0:
        #if float(array[2]) > 1E-8:
            if initial_i == 1:
                count_i = count_i + 1
            ra[source_i].append(float(array[0]))
            dec[source_i].append(float(array[1]))
            flux[source_i].append(float(array[2]))

print(count_i)

137241


In [133]:
file_path = '/Users/z3543600/Downloads/flux_data_fixed/srclist_pumav3_EoR0LoBES_EoR1pietro_CenA-GP_2023-11-07.fits'

with fits.open(file_path) as hdul:
    catalog = hdul[1].data  # Assuming the catalog is in the first extension
    header = hdul[1].header
    hdul.info()

# Create a new FITS table with the updated ra_array and dec_array
new_catalog = catalog.copy()

# Convert to an Astropy Table
table = Table(catalog)
# print(header)

# Print the column names
print(table.colnames)


for source_i in range(10):

    print(source_i)
    
    # Adjust RA values greater than 180
    inds_ra = np.where(new_catalog['ra'] > 180)
    new_catalog['ra'][inds_ra] -= 360

    ra_array = np.array(ra[source_i][1:], dtype=float)
    len_ra_array = len(ra_array)
    print(len_ra_array)
    dec_array = np.array(dec[source_i][1:], dtype=float)
    flux_array = np.array(flux[source_i][1:], dtype=float)
    
    max_ra = np.max(ra_array)
    min_ra = np.min(ra_array)
    if max_ra > 180:
        max_ra -= 360
    if min_ra > 180:
        min_ra -= 360

    max_dec = np.max(dec_array)
    min_dec = np.min(dec_array)
    print(min_ra, max_ra)
    print(min_dec, max_dec)
    
    inds_source_i = np.where((new_catalog['ra'] > min_ra) & (new_catalog['ra'] < max_ra) & 
                             (new_catalog['dec'] > min_dec) & (new_catalog['dec'] < max_dec))
    alpha = np.mean(new_catalog['ALPHA_PL'][inds_source_i])
    print(np.sum(flux_array))
    new_flux = flux_array * (200.0 / 182.435) ** alpha


    inds_keep = np.where((new_catalog['ra'] < min_ra) | (new_catalog['ra'] > max_ra) & 
                        (new_catalog['dec'] < min_dec) | (new_catalog['dec'] > max_dec))
    #inds_keep = np.where((new_catalog['dec'] < min_dec) & (new_catalog['dec'] > max_dec))

    new_catalog['ra'][inds_ra] += 360
    
    new_ids = np.array(['GA' + str(i) for i in range(len(ra_array))])
    new_COMP_TYPE = np.array(['P' for i in range(len(ra_array))])
    new_MOD_TYPE = np.array(['pl' for i in range(len(ra_array))])

    id_array = np.concatenate([new_catalog['UNQ_SOURCE_ID'][inds_keep], new_ids])
    ra_array = np.concatenate([new_catalog['ra'][inds_keep], ra_array])
    dec_array = np.concatenate([new_catalog['dec'][inds_keep], dec_array])
    flux_array = np.concatenate([new_catalog['NORM_COMP_PL'][inds_keep], new_flux])
    alpha_array = np.concatenate([new_catalog['ALPHA_PL'][inds_keep], np.full(len_ra_array, alpha)])
    
    name_array = np.concatenate([new_catalog['NAME'][inds_keep], np.zeros(len_ra_array)])
    MAJOR_DC_array = np.concatenate([new_catalog['MAJOR_DC'][inds_keep], np.zeros(len_ra_array)])
    MINOR_DC_array = np.concatenate([new_catalog['MINOR_DC'][inds_keep], np.zeros(len_ra_array)])
    PA_DC_array = np.concatenate([new_catalog['PA_DC'][inds_keep], np.zeros(len_ra_array)])
    MOD_TYPE_array = np.concatenate([new_catalog['MOD_TYPE'][inds_keep], new_MOD_TYPE])
    COMP_TYPE_array = np.concatenate([new_catalog['COMP_TYPE'][inds_keep], new_COMP_TYPE])
    ALPHA_CPL_array = np.concatenate([new_catalog['ALPHA_CPL'][inds_keep], np.zeros(len_ra_array)])
    CURVE_CPL_array = np.concatenate([new_catalog['CURVE_CPL'][inds_keep], np.zeros(len_ra_array)])

    # Create new FITS columns
    id_col = Column(name='UNQ_SOURCE_ID', data=id_array, format='16A')
    name_col = Column(name='NAME', data=name_array, format='20A')
    ra_col = Column(name='ra', data=ra_array, format='D')
    dec_col = Column(name='dec', data=dec_array, format='D')
    major_dc_col = Column(name='MAJOR_DC', data=MAJOR_DC_array, format='D')
    minor_dc_col = Column(name='MINOR_DC', data=MINOR_DC_array, format='D')
    pa_dc_col = Column(name='PA_DC', data=PA_DC_array, format='D')
    mod_type_col = Column(name='MOD_TYPE', data=MOD_TYPE_array, format='3A')
    comp_type_col = Column(name='COMP_TYPE', data=COMP_TYPE_array, format='1A')
    alpha_col = Column(name='ALPHA_PL', data=alpha_array, format='D')
    flux_col = Column(name='NORM_COMP_PL', data=flux_array, format='D')
    alpha_cpl_col = Column(name='ALPHA_CPL', data=ALPHA_CPL_array, format='D')
    curve_cpl_col = Column(name='CURVE_CPL', data=CURVE_CPL_array, format='D')


    
    # Create a new FITS table with these columns
    new_catalog = Table([id_col, name_col, ra_col, dec_col, major_dc_col, minor_dc_col, pa_dc_col, mod_type_col, comp_type_col, alpha_col, flux_col, alpha_cpl_col, curve_cpl_col])

    
    # new_catalog['UNQ_SOURCE_ID'] = id_array
    # new_catalog['RA'] = ra_array
    # new_catalog['DEC'] = dec_array
    # new_catalog['NORM_COMP_PL'] = flux_array
    # new_catalog['ALPHA_PL'] = alpha_array
    # new_catalog['NAME'] = name_array
    # new_catalog['MAJOR_DC'] = MAJOR_DC_array
    # new_catalog['MINOR_DC'] = MINOR_DC_array
    # new_catalog['PC_DC'] = PA_DC_array
    # new_catalog['MOD_TYPE'] = MOD_TYPE_array
    # new_catalog['COMP_TYPE'] = COMP_TYPE_array
    # new_catalog['ALPHA_CPL'] = ALPHA_CPL_array
    # new_catalog['CURVE_CPL'] = CURVE_CPL_array



    # # Create a new FITS HDU (Header/Data Unit) with the updated data
    # new_hdu = fits.BinTableHDU(data=new_data, header=original_header)
    
    # # Write the new FITS file
    # new_file_path = 'path_to_your_new_fits_file.fits'
    # new_hdu.writeto(new_file_path, overwrite=True)


# Create a new FITS HDU (Header/Data Unit) with the updated data

    # Create a new FITS HDU (Header/Data Unit) with these columns
id_col = fits.Column(name='UNQ_SOURCE_ID', array=id_array, format='16A')
name_col = fits.Column(name='NAME', array=name_array, format='20A')
ra_col = fits.Column(name='RA', array=ra_array, format='D')
dec_col = fits.Column(name='DEC', array=dec_array, format='D')
major_dc_col = fits.Column(name='MAJOR_DC', array=MAJOR_DC_array, format='D')
minor_dc_col = fits.Column(name='MINOR_DC', array=MINOR_DC_array, format='D')
pa_dc_col = fits.Column(name='PA_DC', array=PA_DC_array, format='D')
mod_type_col = fits.Column(name='MOD_TYPE', array=MOD_TYPE_array, format='3A')
comp_type_col = fits.Column(name='COMP_TYPE', array=COMP_TYPE_array, format='1A')
alpha_col = fits.Column(name='ALPHA_PL', array=alpha_array, format='D')
flux_col = fits.Column(name='NORM_COMP_PL', array=flux_array, format='D')
alpha_cpl_col = fits.Column(name='ALPHA_CPL', array=ALPHA_CPL_array, format='D')
curve_cpl_col = fits.Column(name='CURVE_CPL', array=CURVE_CPL_array, format='D')
cols = fits.ColDefs([id_col, name_col, ra_col, dec_col, major_dc_col, minor_dc_col, pa_dc_col, mod_type_col, comp_type_col, alpha_col, flux_col, alpha_cpl_col, curve_cpl_col])
new_hdu = fits.BinTableHDU.from_columns(cols)

# Update the header to match the new columns
new_hdu.header.extend(header, update=True)

print(header)

# Write the new FITS file
new_file_path = '/Users/z3543600/Downloads/flux_data_fixed/srclist_pumav3_EoR0LoBES_EoR1pietro_CenA-GP_2023-11-07_GA40_90_1_smaller.fits'
new_hdu.writeto(new_file_path, overwrite=True)



Filename: /Users/z3543600/Downloads/flux_data_fixed/srclist_pumav3_EoR0LoBES_EoR1pietro_CenA-GP_2023-11-07.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1                1 BinTableHDU     42   348067R x 14C   ['16A', '20A', 'D', 'D', 'D', 'D', 'D', '3A', '1A', 'D', 'D', 'D', 'D', 'D']   
  2                1 BinTableHDU     16   10280R x 4C   [20A, K, K, D]   
['UNQ_SOURCE_ID', 'NAME', 'RA', 'DEC', 'MAJOR_DC', 'MINOR_DC', 'PA_DC', 'MOD_TYPE', 'COMP_TYPE', 'NORM_COMP_PL', 'ALPHA_PL', 'NORM_COMP_CPL', 'ALPHA_CPL', 'CURVE_CPL']
0
12864
5.7515 5.8397
-25.0777 -25.0003
0.07289950799833321
1
15659
3.16425 3.3742500000000013
-28.72525 -28.52575
0.0009246309999469565
2
13139
2.5043333333333333 2.599666666666668
-29.330333333333332 -29.240333333333332
0.0024221557801408495
3
14398
0.8407333333333333 0.9695333333333338
-27.451533333333334 -27.340466666666664
0.003586857033445036
4
17426
-2.3476666666666666 -2.2503333333333

In [123]:
file_path = '/Users/z3543600/Downloads/flux_data_fixed/srclist_pumav3_EoR0LoBES_EoR1pietro_CenA-GP_2023-11-07_GA8_90_1_smaller.fits'

with fits.open(file_path) as hdul:
    catalog = hdul[1].data  # Assuming the catalog is in the first extension
    header = hdul[1].header
    hdul.info()
    print(header)

Filename: /Users/z3543600/Downloads/flux_data_fixed/srclist_pumav3_EoR0LoBES_EoR1pietro_CenA-GP_2023-11-07_GA8_90_1_smaller.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1                1 BinTableHDU     34   352335R x 13C   ['16A', '20A', 'D', 'D', 'D', 'D', 'D', '3A', '1A', 'D', 'D', 'D', 'D']   
XTENSION= 'BINTABLE'           / binary table extension                         BITPIX  =                    8 / array data type                                NAXIS   =                    2 / number of array dimensions                     NAXIS1  =                  112 / length of dimension 1                          NAXIS2  =               352335 / length of dimension 2                          PCOUNT  =                    0 / number of group parameters                     GCOUNT  =                    1 / number of groups                               TFIELDS =                   13 / number of table fields          